#  Análisis y comparación de modelos SVC para predicción de room_type de Airbnb en Madrid

## Info del dataset

In [1]:
# Cargar el dataset
import pandas as pd
df = pd.read_csv('../data/airbnb.csv')
df

,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365
0,Centro,Justicia,40.424715,-3.698638,Entire home/apt,49,28,35,0.42,1,99
1,Centro,Embajadores,40.413418,-3.706838,Entire home/apt,80,5,18,0.30,1,188
2,Moncloa - Aravaca,Argüelles,40.424920,-3.713446,Entire home/apt,40,2,21,0.25,9,195
3,Moncloa - Aravaca,Casa de Campo,40.431027,-3.724586,Entire home/apt,55,2,3,0.13,9,334
4,Latina,Cármenes,40.403410,-3.740842,Private room,16,2,23,0.76,2,250
...,...,...,...,...,...,...,...,...,...,...,...
13316,Centro,Justicia,40.427500,-3.698354,Private room,14,1,0,0.00,1,10
13317,Chamberí,Gaztambide,40.431187,-3.711909,Entire home/apt,47,1,0,0.00,7,354
13318,Centro,Palacio,40.413552,-3.711461,Entire home/apt,60,2,0,0.00,1,17
13319,Centro,Universidad,40.425400,-3.709921,Entire home/apt,150,5,0,0.00,1,15


## LinearSVC vs SVC(rbf) - Template de modelado y evaluación

In [3]:
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.svm import LinearSVC, SVC
import warnings
warnings.filterwarnings('ignore')

# =========== CARGA DE DATOS ============
df = pd.read_csv('../data/airbnb.csv')
print(f"Filas iniciales: {len(df)}")

# =========== SAMPLE DE LOS DATOS ============
# Muestreo para acelerar el proceso
df = df.sample(n=13321, random_state=42) # Filas totales originales 13321
print(f"Filas después del muestreo: {len(df)}")
print(f"Columnas después del sample: {df.columns.tolist()}")

# ============ PREPROCESAMIENTO BÁSICO ============

# 2. Eliminar outliers en variables clave
outlier_cols = ['price', 'minimum_nights', 'calculated_host_listings_count']

Q1 = df[outlier_cols].quantile(0.25)
Q3 = df[outlier_cols].quantile(0.75)
IQR = Q3 - Q1

masks = [
    (df[c] >= (Q1[c] - 1.5 * IQR[c])) &
    (df[c] <= (Q3[c] + 1.5 * IQR[c]))
    for c in outlier_cols
]
mask = np.logical_and.reduce(masks)

before = len(df)
df = df[mask].copy()
print(f"Filas antes: {before}, después de eliminar outliers: {len(df)}")

# 3. Eliminar columnas de latitud y longitud
df = df.drop(columns=['latitude', 'longitude'], errors='ignore')
print(f"Columnas después de eliminar lat/long: {df.columns.tolist()}")

# 4. Eliminar la columna de neighbourhood (alta cardinalidad)
df = df.drop(columns=['neighbourhood'], errors='ignore')
print(f"Columnas después de eliminar neighbourhood: {df.columns.tolist()}")

# ============ PREPARACIÓN DE DATOS ============
X = df.drop(columns=['room_type'])
y = df['room_type'].values

# Codificar target
le = LabelEncoder()
y_encoded = le.fit_transform(y)

print(f"\nDistribución de room_type:\n{pd.Series(y).value_counts()}")
print(f"\nClases codificadas: {le.classes_}")

# Columnas numéricas
num_cols = ['price', 'minimum_nights', 'number_of_reviews',
            'reviews_per_month', 'calculated_host_listings_count', 'availability_365']

# Columnas categóricas
cat_cols = ['neighbourhood_group']

preprocessor = ColumnTransformer([
    ('num', StandardScaler(), num_cols),
    ('cat', OneHotEncoder(handle_unknown='ignore', drop='first'), cat_cols)
], remainder='drop')

cv = KFold(n_splits=5, shuffle=True, random_state=42)

# ============ FUNCIÓN DE EVALUACIÓN ============
def cv_metrics(estimator, X, y_encoded, cv):
    """Evaluación con validación cruzada"""
    from sklearn.base import clone

    accs, f1s, precs, recs = [], [], [], []

    for train_idx, val_idx in cv.split(X):
        est = clone(estimator)
        X_tr, X_val = X.iloc[train_idx], X.iloc[val_idx]
        y_tr, y_val = y_encoded[train_idx], y_encoded[val_idx]

        est.fit(X_tr, y_tr)
        y_pred = est.predict(X_val)

        accs.append(accuracy_score(y_val, y_pred))
        f1s.append(f1_score(y_val, y_pred, average='macro'))
        precs.append(precision_score(y_val, y_pred, average='macro'))
        recs.append(recall_score(y_val, y_pred, average='macro'))

    return {
        'accuracy': np.mean(accs),
        'f1_score': np.mean(f1s),
        'precision': np.mean(precs),
        'recall': np.mean(recs)
    }

# ============ PIPE LinearSVC ============
print("\n" + "="*70)
print("  EVALUANDO LinearSVC")
print("="*70)

pipe_linear = Pipeline([
    ('pre', preprocessor),
    ('model', LinearSVC(max_iter=10000, random_state=42))
])

results_linear = cv_metrics(pipe_linear, X, y_encoded, cv)

# ============ PIPE SVC (RBF) ============
print("\n" + "="*70)
print("  EVALUANDO SVC(RBF)")
print("="*70)

pipe_svc = Pipeline([
    ('pre', preprocessor),
    ('model', SVC(kernel='rbf', random_state=42))
])

results_svc = cv_metrics(pipe_svc, X, y_encoded, cv)

# ============ COMPARACIÓN DE RESULTADOS ============
print('\n' + "="*70)
print("  RESULTADOS")
print("="*70)

comparison = pd.DataFrame({
    'Modelo': ['LinearSVC', 'SVC(RBF)'],
    'Accuracy': [results_linear['accuracy'], results_svc['accuracy']],
    'F1-Score': [results_linear['f1_score'], results_svc['f1_score']],
    'Precision': [results_linear['precision'], results_svc['precision']],
    'Recall': [results_linear['recall'], results_svc['recall']]
})

print(f"\n{comparison.to_string(index=False)}")

print("\n" + "="*70)
print("  CONCLUSIÓN")
print("="*70)

if results_svc['accuracy'] > results_linear['accuracy']:
    print("\n✅ SVC(RBF) muestra mejor rendimiento que LinearSVC")
    print("   → Proceder con afinación de hiperparámetros en SVC(RBF)")
else:
    print("\n⚠️  LinearSVC muestra mejor o igual rendimiento")
    print("   → Los datos podrían ser linealmente separables")

print(f"\n💡 Mejora de SVC(RBF) sobre LinearSVC:")
print(f"   • Accuracy: {((results_svc['accuracy'] - results_linear['accuracy']) / results_linear['accuracy'] * 100):+.2f}%")
print(f"   • F1-Score: {((results_svc['f1_score'] - results_linear['f1_score']) / results_linear['f1_score'] * 100):+.2f}%")

Filas iniciales: 13321
Filas después del muestreo: 13321
Columnas después del sample: ['neighbourhood_group', 'neighbourhood', 'latitude', 'longitude', 'room_type', 'price', 'minimum_nights', 'number_of_reviews', 'reviews_per_month', 'calculated_host_listings_count', 'availability_365']
Filas después de eliminar duplicados: 13321
Filas antes: 13321, después de eliminar outliers: 10318
Columnas después de eliminar lat/long: ['neighbourhood_group', 'neighbourhood', 'room_type', 'price', 'minimum_nights', 'number_of_reviews', 'reviews_per_month', 'calculated_host_listings_count', 'availability_365']
Columnas después de eliminar neighbourhood: ['neighbourhood_group', 'room_type', 'price', 'minimum_nights', 'number_of_reviews', 'reviews_per_month', 'calculated_host_listings_count', 'availability_365']

Distribución de room_type:
Entire home/apt    5764
Private room       4411
Shared room         143
Name: count, dtype: int64

Clases codificadas: ['Entire home/apt' 'Private room' 'Shared roo

## SVC(rbf) - FineTuning + DataTreatment


In [8]:
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.svm import SVC
from sklearn.metrics import (
    accuracy_score, f1_score, precision_score, recall_score,
    classification_report, confusion_matrix
)
from sklearn.base import BaseEstimator, TransformerMixin, clone
from haversine import haversine
import warnings
warnings.filterwarnings('ignore')

# =============================================================================
# CUSTOM TRANSFORMER FOR FEATURE ENGINEERING
# =============================================================================
class FeatureEngineer(BaseEstimator, TransformerMixin):
    """Custom transformer for feature engineering, avoiding data leakage."""
    def __init__(self):
        self.sol_coords = (40.416775, -3.703790)  # Puerta del Sol, Madrid

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X = X.copy()
        # Distance to city center (geographical feature)
        X['distance_to_center'] = X.apply(
            lambda row: haversine(self.sol_coords, (row['latitude'], row['longitude'])),
            axis=1
        )
        # Price category segmentation
        X['price_category'] = pd.cut(
            X['price'],
            bins=[0, 30, 60, 100, float('inf')],
            labels=['budget', 'moderate', 'premium', 'luxury']
        )
        # Drop original coordinates after feature creation
        X = X.drop(columns=['latitude', 'longitude'], errors='ignore')
        return X

# =============================================================================
# DATA LOADING AND PREPROCESSING
# =============================================================================
def load_and_preprocess_data():
    df = pd.read_csv('../data/airbnb.csv')
    print(f"Initial rows: {len(df):,}")
    df = df.sample(n=13321, random_state=42)
    print(f"Rows after sampling: {len(df):,}")

    # Outlier filtering
    q1, q3 = df['price'].quantile([0.05, 0.95])
    iqr = q3 - q1
    lower, upper = q1 - 1.5 * iqr, q3 + 1.5 * iqr
    df = df[(df['price'] >= lower) & (df['price'] <= upper)].copy()
    df = df[df['minimum_nights'] <= 365].copy()
    df = df[df['calculated_host_listings_count'] <= df['calculated_host_listings_count'].quantile(0.95)].copy()
    print(f"Rows after outlier filtering: {len(df):,}")

    # Feature selection and zero value handling
    df = df.drop(columns=['neighbourhood'], errors='ignore')
    for col in ['number_of_reviews', 'reviews_per_month', 'availability_365']:
        df[col] = df[col].replace(0, np.nan)

    return df

# =============================================================================
# MODEL PIPELINE AND EVALUATION
# =============================================================================
def build_pipeline():
    # Feature groups
    num_cols_base = ['price', 'minimum_nights', 'reviews_per_month',
                     'calculated_host_listings_count', 'availability_365']
    num_cols_engineered = ['distance_to_center']
    cat_cols = ['neighbourhood_group', 'price_category']

    # Transformers
    numeric_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='median')),
        ('scaler', StandardScaler())
    ])

    categorical_transformer = Pipeline(steps=[
        ('encoder', OneHotEncoder(
            handle_unknown='ignore',
            drop='first',
            min_frequency=0.01
        ))
    ])

    preprocessor = ColumnTransformer(
        transformers=[
            ('num_base', numeric_transformer, num_cols_base),
            ('num_eng', numeric_transformer, num_cols_engineered),
            ('cat', categorical_transformer, cat_cols)
        ],
        remainder='drop'
    )

    # Full pipeline
    pipeline = Pipeline([
        ('feature_eng', FeatureEngineer()),
        ('preprocessor', preprocessor),
        ('model', SVC(kernel='rbf', class_weight='balanced', cache_size=6389, random_state=42))
    ])

    return pipeline

def evaluate_model_cv(pipeline, X, y_encoded, cv, le):
    """Cross-validated model evaluation without data leakage."""
    accs, f1s, precs, recs = [], [], [], []
    for fold, (train_idx, val_idx) in enumerate(cv.split(X, y_encoded), 1):
        X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
        y_train, y_val = y_encoded[train_idx], y_encoded[val_idx]
        model = clone(pipeline)
        model.fit(X_train, y_train)
        y_pred = model.predict(X_val)
        accs.append(accuracy_score(y_val, y_pred))
        f1s.append(f1_score(y_val, y_pred, average='macro'))
        precs.append(precision_score(y_val, y_pred, average='macro'))
        recs.append(recall_score(y_val, y_pred, average='macro'))
        print(f"  Fold {fold}: Accuracy={accs[-1]:.4f}, F1={f1s[-1]:.4f}, Precision={precs[-1]:.4f}, Recall={recs[-1]:.4f}")

    return {
        'accuracy_mean': np.mean(accs),
        'accuracy_std': np.std(accs),
        'f1_mean': np.mean(f1s),
        'f1_std': np.std(f1s),
        'precision_mean': np.mean(precs),
        'precision_std': np.std(precs),
        'recall_mean': np.mean(recs),
        'recall_std': np.std(recs)
    }

# =============================================================================
# MAIN EXECUTION
# =============================================================================
def main():
    # Data loading and preprocessing
    df = load_and_preprocess_data()

    # Target encoding
    X = df.drop(columns=['room_type'])
    y = df['room_type'].values
    le = LabelEncoder()
    y_encoded = le.fit_transform(y)
    print(f"\nClass distribution:\n{pd.Series(y).value_counts()}")
    print(f"\nClasses: {le.classes_}")

    # Model pipeline
    pipeline = build_pipeline()
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

    # Hyperparameter tuning
    param_grid = {
        'model__C': np.logspace(-2, 3, 6),
        'model__gamma': np.logspace(-4, 1, 6)
    }
    print(f"\nHyperparameter search space: {len(param_grid['model__C'])} x {len(param_grid['model__gamma'])} = {len(param_grid['model__C']) * len(param_grid['model__gamma'])} combinations")

    grid_search = GridSearchCV(
        pipeline,
        param_grid,
        cv=cv,
        scoring='f1_macro',
        n_jobs=-1,
        verbose=1,
        return_train_score=True
    )
    print("\nTraining model with GridSearchCV...")
    grid_search.fit(X, y_encoded)

    # Results
    print(f"\nBest parameters: {grid_search.best_params_}")
    print(f"Best F1 score (CV): {grid_search.best_score_:.4f}")

    # Final evaluation
    print("\nFinal model evaluation (cross-validated):")
    best_pipeline = grid_search.best_estimator_
    metrics = evaluate_model_cv(best_pipeline, X, y_encoded, cv, le)

    print("\nFinal performance metrics:")
    print(f"Accuracy:  {metrics['accuracy_mean']:.4f} ± {metrics['accuracy_std']:.4f}")
    print(f"F1-Score:  {metrics['f1_mean']:.4f} ± {metrics['f1_std']:.4f}")
    print(f"Precision: {metrics['precision_mean']:.4f} ± {metrics['precision_std']:.4f}")
    print(f"Recall:    {metrics['recall_mean']:.4f} ± {metrics['recall_std']:.4f}")

    # Classification report
    print("\nClassification Report:")
    best_pipeline.fit(X, y_encoded)
    y_pred_final = best_pipeline.predict(X)
    print(classification_report(y_encoded, y_pred_final, target_names=le.classes_))

    # Confusion matrix
    print("\nConfusion Matrix:")
    cm = confusion_matrix(y_encoded, y_pred_final)
    print(cm)
    print(f"\nClass order: {le.classes_}")

    # Summary
    print("\n" + "="*70)
    print("EXECUTIVE SUMMARY")
    print("="*70)
    print(f"""
    Optimized SVC(RBF) Model:
      • C = {grid_search.best_params_['model__C']:.4f}
      • gamma = {grid_search.best_params_['model__gamma']:.6f}

    Performance:
      • Accuracy:  {metrics['accuracy_mean']:.4f} ± {metrics['accuracy_std']:.4f}
      • F1-Score:  {metrics['f1_mean']:.4f} ± {metrics['f1_std']:.4f}
      • Precision: {metrics['precision_mean']:.4f} ± {metrics['precision_std']:.4f}
      • Recall:    {metrics['recall_mean']:.4f} ± {metrics['recall_std']:.4f}
    """)
    print("="*70)

if __name__ == "__main__":
    main()


Initial rows: 13,321
Rows after sampling: 13,321
Rows after outlier filtering: 12,549

Class distribution:
Entire home/apt    7320
Private room       5037
Shared room         192
Name: count, dtype: int64

Classes: ['Entire home/apt' 'Private room' 'Shared room']

Hyperparameter search space: 6 x 6 = 36 combinations

Training model with GridSearchCV...
Fitting 5 folds for each of 36 candidates, totalling 180 fits

Best parameters: {'model__C': np.float64(10.0), 'model__gamma': np.float64(1.0)}
Best F1 score (CV): 0.6799

Final model evaluation (cross-validated):
  Fold 1: Accuracy=0.8701, F1=0.7122, Precision=0.6847, Recall=0.7720
  Fold 2: Accuracy=0.8733, F1=0.6656, Precision=0.6562, Recall=0.6801
  Fold 3: Accuracy=0.8693, F1=0.6643, Precision=0.6568, Recall=0.6748
  Fold 4: Accuracy=0.8633, F1=0.6652, Precision=0.6642, Recall=0.6679
  Fold 5: Accuracy=0.8601, F1=0.6923, Precision=0.6744, Recall=0.7218

Final performance metrics:
Accuracy:  0.8672 ± 0.0048
F1-Score:  0.6799 ± 0.0193